In [1]:
using JSON
using HomotopyContinuation
using Pandas

In [3]:
function read_and_parse_json(filename::String)
    raw_content = read(filename, String)
    parsed_content = JSON.parse(raw_content)
    return parsed_content
end;

In [4]:
parse_polynomialstr = x -> begin
    exprforparse = replace(x, "**"=> "^")  
    exprforparse = replace(exprforparse, r"x_(\d+)" => 
    m -> "x[" * string(parse(Int, match(r"x_(\d+)", m).captures[1]) + 1) * "]")
    return eval(Meta.parse(exprforparse))
end;

In [5]:
content = read_and_parse_json("../../applications/data/polyproblems_v2_small.json");

In [9]:
test = "(10, 15, 1, 0, 0)"
str, data = test, content[test]
idx = 0
str = replace(str, "(" => "", ")" => "")  # Remove parentheses
strs = split(str, ",")  # Split the string by commas
probinfo = parse.(Int, strs)  # Parse each string into a number
mlen,nlen,_,_,_ = probinfo
functional_sets = data["functional_sets"]
objective = data["objective"]
eqs = [fset["residual"] for fset in functional_sets]
@var x[0:nlen-1];
polynomials = [parse_polynomialstr(expr) for expr in eqs]
obj = parse_polynomialstr(objective)
J = differentiate(polynomials, x);
dfdx = differentiate(obj, x);
@var l[1:mlen];
kkt = [dfdx + J'*l; polynomials];
Opt = System(kkt, variables = [x; l]);
# solver, starts = solver_startsolutions(Opt);
solve(Opt)
# real_sols = real_solutions(res);
# println("Real solutions: ", probinfo, " => ", length(real_sols))
# if length(real_sols) > 0
#     minarg = (real_sols .|> y->evaluate(obj, x=>y[1:nlen])) |> argmin |> y->real_sols[y];
#     println("local opt ; min = ", evaluate(obj, x=>minarg[1:nlen]))
# end
# errir with probinfo
#println("Error: ", probinfo)

Computing mixed cells... 56    Time: 0:00:01
  mixed_volume:  1100


LoadError: InexactError: trunc(Int32, 92945905954898239)

In [16]:
res

LoadError: UndefVarError: `res` not defined

In [28]:
real_sols = real_solutions(res);

In [32]:
obj

(-1 + x₁)^2 + (-1 + x₁₂)^2 + (-1 + x₁₄)^2 + (-1 + x₄)^2 + (-1 + x₅)^2

In [29]:
sols_and_obj = real_sols .|> y -> [y; evaluate(obj, x=>y[1:nlen])]
DataFrame(permutedims(hcat(sols_and_obj...))')

,0,1,2,3,4,5
0,-9.371429e-01,-9.522963e-01,-9.081918e-01,-9.155556e-01,-8.977778e-01,-9.584749e-01
1,-3.111111e+00,-6.250000e-01,4.714045e-01,6.666667e-01,3.333333e-01,-4.714045e-01
2,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
3,-6.830000e+01,-1.855750e+01,1.553087e+01,2.010000e+01,1.230000e+01,-6.530866e+00
4,4.187692e+00,1.822613e+01,6.409132e+00,5.844103e+00,7.208205e+00,2.550868e+00
5,1.820000e+01,1.125000e+00,-2.757716e+00,-3.900000e+00,-1.950000e+00,2.757716e+00
6,-7.142857e-02,-3.555556e-01,4.714045e-01,3.333333e-01,6.666667e-01,-4.714045e-01
7,-5.952381e-01,-5.952381e-01,-5.952381e-01,-5.952381e-01,-5.952381e-01,-5.952381e-01
8,-2.857143e-01,-2.857143e-01,-2.857143e-01,-2.857143e-01,-2.857143e-01,-2.857143e-01
9,0.000000e+00,2.821186e-37,5.345529e-51,0.000000e+00,-7.174648e-43,-3.085099e-41


In [30]:
minarg = (real_sols .|> y->evaluate(obj, x=>y[1:nlen])) |> argmin |> y->real_sols[y];
println("Real solutions: ", probinfo, " => ", length(real_sols), "; objval= ", evaluate(obj, x=>minarg[1:nlen]))

Real solutions: [10, 15, 1, 4, 2] => 6; objval= 12.539460198869095


In [17]:
for (idx, (str, data)) in enumerate(content)
    str = replace(str, "(" => "", ")" => "")  # Remove parentheses
    strs = split(str, ",")  # Split the string by commas
    probinfo = parse.(Int, strs)  # Parse each string into a number
    mlen,nlen,_,_,_ = probinfo
    if mlen == 10
        functional_sets = data["functional_sets"]
        objective = data["objective"]
        eqs = [fset["residual"] for fset in functional_sets]
        @var x[1:nlen]
        polynomials = [parse_polynomialstr(expr) for expr in eqs]
        obj = parse_polynomialstr(objective)
        J = differentiate(polynomials, x);
        dfdx = differentiate(obj, x);
        @var l[1:mlen];
        println(mlen, nlen)
        kkt = [dfdx + J'*l; polynomials];
        try
            Opt = System(kkt, variables = [x; l]);
            res = solve(Opt)
            real_sols = real_solutions(res);
            minarg = (real_sols .|> y->evaluate(obj, x=>y[1:nlen])) |> argmin |> y->real_sols[y];
            println("Real solutions: ", probinfo, " => ", length(real_sols), "local opt ; min = ", evaluate(obj, x=>minarg[1:nlen]))
        catch e
            # errir with probinfo
            println("Error: ", probinfo)
        end
    end
end

Computing mixed cells... 1475    Time: 0:01:45


In [70]:
content = read_and_parse_json("../../applications/data/testproblems.json")
key = "(10, 15, 1, 2, 0)"
value = content[key]
functional_sets = value["functional_sets"]
objective = value["objective"]
eqs = [fset["residual"] for fset in functional_sets];

In [71]:
nlen = 15
mlen = 10;

In [72]:
@var x[1:nlen];

In [73]:
polynomials = [parse_polynomialstr(expr) for expr in eqs]

exprforparse = replace(objective, "**"=> "^")   
exprforparse = replace(exprforparse, r"x_(\d+)" => m -> "x[" * string(parse(Int, match(r"x_(\d+)", m).captures[1]) + 1) * "]")
obj = eval(Meta.parse(exprforparse))

(-1 + x₁)^2 + (-1 + x₁₁)^2 + (-1 + x₁₂)^2 + (-1 + x₁₄)^2 + (-1 + x₃)^2

In [74]:
vars = x[1:nlen];

In [75]:
J = differentiate(polynomials, vars);
dfdx = differentiate(obj, vars);

In [76]:
@var l[1:length(polynomials)];

In [77]:
kkt = [dfdx + J'*l; polynomials];

In [78]:
try
    Opt = System(kkt, variables = [vars; l]);
    res = solve(Opt)
catch e
    res = e
end

Tracking 730 paths... 100%|█████████████████████████████| Time: 0:00:07
  # paths tracked:                  730
  # non-singular solutions (real):  0 (0)
  # singular endpoints (real):      294 (0)
  # total solutions (real):         294 (0)


Result with 294 solutions
• 730 paths tracked
• 0 non-singular solutions (0 real)
• 294 singular solutions (0 real)
• random_seed: 0xcbb7a012
• start_system: :polyhedral
• multiplicity table of singular solutions:
╭───────┬───────┬────────┬────────────╮
│ mult. │ total │ # real │ # non-real │
├───────┼───────┼────────┼────────────┤
│   1   │  294  │   0    │    294     │
╰───────┴───────┴────────┴────────────╯


In [79]:
real_sols = real_solutions(res);
sols_and_obj = real_sols .|> x -> [x[1:nlen]; evaluate(obj, vars=>x[1:nlen])]
minarg = (real_sols .|> x->evaluate(obj, vars=>x[1:nlen])) |> argmin |> x->real_sols[x];

LoadError: MethodError: reducing over an empty collection is not allowed; consider supplying `init` to the reducer

In [ ]:
DataFrame(permutedims(hcat(sols_and_obj...))')

,0,1,2,3,4,5,6,7,8,9
0,-0.155826,0.283759,-0.841618,-0.569985,-4.031184,-6.697917,0.390150,-3.685179,1.024392,0.081983
1,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,-0.172620,20.257550,-0.181014,-0.682777,10.881048,12.384704,-15.405195,-16.079478,-0.783762,-0.157914
3,1.215208,0.820810,-0.387527,-0.336030,2.467960,-1.066667,0.933570,2.452419,-1.066667,1.207592
4,-0.707555,-0.597931,0.845351,0.653891,-0.555033,1.000000,-0.359274,-0.225784,1.000000,-0.350338
5,-1.113804,-1.474237,-1.804451,2.203295,-0.882442,1.000000,1.783315,1.387938,1.000000,1.587854
6,-0.157041,11.782274,0.314533,0.362158,1.137384,-3.789336,7.446773,1.061461,-0.537924,0.070726
7,0.350609,-0.638458,1.893642,1.282467,9.070164,15.070314,-0.877837,8.291653,-2.304882,-0.184461
8,0.428672,-67.095921,-9.225452,13.385162,-0.790790,0.543409,50.762521,0.560185,-3.553040,0.111620
9,4.278267,-2.349412,0.792125,1.169621,0.165377,0.099533,-1.708746,0.180905,-0.650793,-8.131807


In [39]:
minarg[nlen+1:end]

10-element Vector{Float64}:
  1.793662034335766e-42
  1.9132431525062344e-42
  8.609577764811676e-42
 -2.8698592549372254e-41
 -2.8698592549372254e-41
  1.1479437019748901e-41
 -2.7369110631344083e-48
 -2.0089014784560578e-41
 -1.3684555315672042e-48
 -8.609577764811676e-42

In [40]:
evaluate.(polynomials, vars=> minarg[1:nlen])

10-element Vector{Float64}:
  0.0
  0.0
 -4.440892098500626e-16
  0.0
  0.0
  0.0
 -1.1102230246251565e-16
  2.220446049250313e-16
  0.0
  4.440892098500626e-16

# Groebner bases

In [ ]:
using DynamicPolynomials
using Groebner

In [4]:
@polyvar x[0:15];

In [6]:
polynomials = Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}()

for expr in eqs
    exprforparse = replace(expr, r"0\."=> "")  
    exprforparse = replace(exprforparse, r"x_(\d+)" => 
    m -> "x[" * string(parse(Int, match(r"x_(\d+)", m).captures[1]) + 1) * "]")
    # println(exprforparse)
    poly = eval(Meta.parse(exprforparse))
    # println([poly])
    push!(polynomials, poly)
end

In [7]:
polynomials

10-element Vector{Polynomial{DynamicPolynomials.Commutative{DynamicPolynomials.CreationOrder}, Graded{LexOrder}, Int64}}:
 8 + 6x₃x₁₀ - x₂x₉
 7 - 9x₄x₈ - 6x₃x₁₄ + 5x₀x₉x₁₃
 -8 - 9x₄x₁₄ - 2x₀x₅
 1 - 2x₀ + 2x₂x₈x₁₃
 4 + 3x₁x₂x₁₀
 -4 + 4x₃x₈ + x₀x₇
 -3 + 9x₉x₁₀x₁₁
 -5 + 8x₈x₁₀x₁₂ - 3x₆x₉x₁₃
 -5 + 6x₈ + 7x₄x₁₀
 -1 - 3x₉x₁₄ - 7x₀x₆

In [8]:
groebner(polynomials)